# Theft Crime Prediction using KDE Tweets in Cicago

In [1]:
%matplotlib inline

import os
import glob
import itertools
import functools
import collections

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm_notebook, tqdm

from utils.consts import START_DATE, END_DATE, \
                         RAW_CRIMES_DATA_PATH, PROCESSED_CRIMES_DATA_PATH, \
                         RAW_TWEETS_DATA_WILDCARD_PATH, PROCESSED_TWEETS_DATA_PATH, \
                         CSV_DATE_FORMART

from utils.lda import print_top_words_LDA, print_top_words_LDA, get_topic_top_words_LDA

from utils.surveillance import generate_all_data_surveillance_data, \
                               generate_one_step_datasets, generate_surveillance_data, calc_AUCs
    
from utils.visualization import plot_contour, plot_scatter, plot_imshow, plot_log_reg_coef, plot_surveillance_data


In [2]:
import warnings
warnings.simplefilter('ignore')

## Pipline Diagram

![title](./ANLP-Project-Pipeline.png)

## Research Time Frame

In [3]:
print(START_DATE, '--->', END_DATE)

2017-12-08 ---> 2018-02-19


## Data Sources & Preprocessing

### Chicago Crimes Incidents (only THEFT)

In [4]:
if not os.path.exists(PROCESSED_CRIMES_DATA_PATH):
    !python3 ./preprocess_crimes_data.py {RAW_CRIMES_DATA_PATH}  {PROCESSED_CRIMES_DATA_PATH}

In [5]:
crimes_data = pd.read_csv(PROCESSED_CRIMES_DATA_PATH)
crimes_data['timestamp'] = pd.to_datetime(crimes_data['timestamp'], format=CSV_DATE_FORMART).dt.normalize()

In [6]:
len(crimes_data)

10902

In [7]:
crimes_data['timestamp'].agg(['min', 'max'])

min   2017-12-08
max   2018-02-19
Name: timestamp, dtype: datetime64[ns]

### Tweets

In [8]:
if not os.path.exists(PROCESSED_TWEETS_DATA_PATH):
    !python3 -W ignore ./preprocess_tweets_data.py {RAW_TWEETS_DATA_WILDCARD_PATH} {PROCESSED_TWEETS_DATA_PATH}

In [9]:
tweets_data = pd.read_csv(PROCESSED_TWEETS_DATA_PATH)
tweets_data['timestamp'] = pd.to_datetime(tweets_data['timestamp'], format=CSV_DATE_FORMART).dt.normalize()
tweets_data['tokens'] = tweets_data['tokens'].apply(lambda x: eval(x))

In [10]:
len(tweets_data)

79634

In [11]:
tweets_data['timestamp'].agg(['min', 'max'])

min   2017-12-08
max   2018-02-19
Name: timestamp, dtype: datetime64[ns]

### Enreaching every Tweet with Sentiment Analysis

For **demonstration propuse**, the training and evaluation of the Sentiment Analysis Model is done every time the notebook is executed.
The same goes for calculating the sentiment value for each tweet.

In [ ]:
%%time

from utils.sentiment.sentiment import calculate_sentiment_tweet

Length of the testing Corpus : 10662
Adding unigrams and bigrams sentiment scores 

average fit_time : 0.42835249900817873
average score_time : 0.007321763038635254
average test_f1_micro : 0.7554900886087512
average test_f1_macro : 0.7554053496579932
average test_precision_micro : 0.7554900886087512
average test_precision_macro : 0.7558688268181593
average test_recall_micro : 0.7554900886087512
average test_recall_macro : 0.7554900886087512
CPU times: user 8.3 s, sys: 315 ms, total: 8.62 s
Wall time: 9.47 s


In [ ]:
%%time

tweets_data['sentiment'] = tweets_data['tokens'].apply(lambda x: calculate_sentiment_tweet(' '.join(x)))

## Detailed analysis of 31 days training dataset and the successive single day for evaluation

### Generating the train dataset with all the features (KDA, SENTIMENT, LDA) and the evaluation dataset (successive day crime incidents)

In [ ]:
%%time

train_dataset, evaluation_dataset = generate_one_step_datasets(crimes_data,
                                                               tweets_data,
                                                               START_DATE,
                                                               31)

#### KDE

In [ ]:
plot_scatter(train_dataset['X'][train_dataset['Y']][['latitude', 'longitude']])
plt.title('Crime Incidents: {}+31 days', START_DATE)

In [ ]:
plot_contour(train_dataset['KDE'])
plt.title('KDE of Crime Incidents: {}+31 days', START_DATE)

#### Sentiment

In [ ]:
sns.distplot(train_dataset['SENTIMENT'], norm_hist=True)
plt.xlabel('sentiment value')
plt.ylabel('sentiment count')
plt.title('Sentiment Histogram & Distribution over Geo Documents')

#### LDA

In [ ]:
print_top_words_LDA(train_dataset['LDA']['model'], train_dataset['LDA']['vocabulary'], 5)

Lets try to examine one "interesting" topic vector. First we check which ones have the maximal variance in its values.

In [ ]:
np.argsort(train_dataset['LDA']['model'].components_.std(axis=1))[-5:]

Lets pick up the maximal one in this sense.

In [ ]:
exmple_topic_id = np.argsort(train_dataset['LDA']['model'].components_.std(axis=1))[-1]
print(exmple_topic_id)
example_topic_column_name = 'T{:03}'.format(exmple_topic_id)

In [ ]:
get_topic_top_words_LDA(exmple_topic_id, train_dataset['LDA']['model'], train_dataset['LDA']['vocabulary'], 15)

## Crime Prediction

In [ ]:
surveillance_data, threat_datasets = generate_surveillance_data(train_dataset,
                                                                evaluation_dataset)

### Logic Regression Coefficients
*Note: All features were standatarized, therefore there is a meaning for coefficeints comparing.*

#### SENTIMENT model

In [ ]:
plot_log_reg_coef(threat_datasets, 'KDE+SENTIMENT')


#### KDE+LDA model

In [ ]:
plot_log_reg_coef(threat_datasets, 'KDE+LDA')

#### KDE+SENTIMENT+LDA model

In [ ]:
most_dominant_coefs_index = plot_log_reg_coef(threat_datasets, 'KDE+SENTIMENT+LDA') - 2

Examining the top words for the maximal variance topic:

In [ ]:
get_topic_top_words_LDA(most_dominant_coefs_index,
                        train_dataset['LDA']['model'], train_dataset['LDA']['vocabulary'], 15)


### Threat Maps

In [ ]:
plot_imshow(threat_datasets['KDE']['df'], 'KDE')




In [ ]:
# plot_imshow(threat_datasets['SENTIMENT']['df'], 'SENTIMENT')




In [ ]:
# plot_imshow(threat_datasets['LDA']['df'], 'LDA')


In [ ]:
# plot_imshow(threat_datasets['SENTIMENT+LDA']['df'], 'SENTIMENT+LDA')


### Surveillance Plot & AUC

In [ ]:
normalized_surveillance_data = surveillance_data.cumsum(axis=1) / surveillance_data.sum(axis=1)[:, None]

In [ ]:
plot_surveillance_data(normalized_surveillance_data, threat_datasets.keys())

In [ ]:
print('AUC betwwen models')
calc_AUCs(normalized_surveillance_data, threat_datasets.keys())

## Full Training & Evaluation over all the period (~60 steps of 31 days)
**TAKES ~3 HOURS, CHANGE THE VARIABLE `is_full_run` TO `True` IN ORDER TO PREFORM IT**

In [ ]:
is_full_run = True

In [ ]:
if is_full_run:
    agg_surveillance_data, all_threat_datasets = generate_all_data_surveillance_data(crimes_data, tweets_data, 31)   

In [ ]:
if is_full_run:
    plot_surveillance_data(agg_surveillance_data, all_threat_datasets[0][1].keys())

In [ ]:
if is_full_run:
    print('AUC betwwen models')
    calc_AUCs(agg_surveillance_data, all_threat_datasets[0][1].keys())